# Scaleway AI Inference Quick Start : Simple Machine Learning example

How to use:
- Execute cells with `SHIFT + ENTER``
- Make sure to execute cells in order (Executing previous cells might fails). 
- In case you want to reset the environment, use the menu "Kernel", "Restart Kernel and clear all outputs" and restart cells executionfrom the begining

## About AI Inference

Scaleway AI Inference provides clients with a flexible Machine Learning (ML) inference function which runs a trained model for clients on managed infrastructure and with built-in scalability.

By taking advantage of the serverless architecture, we provide:
- An adapted runtime for your model to perform fast predictions.
- A predictable billing that you pay only when predictions are made. The serveless function is only up when you call it.
- Auto-Scalability makes the inference server adapt to handle as much requests as you send it.The Scaleway Inference product provides clients with a flexible Machine Learning (ML) inference function which runs a trained model for clients on managed infrastructure and with built-in scalability.


### About this demo

In this demo, the [Scaleway AI Inference API](https://developers.scaleway.com/en/products/inference/api/v1alpha1/) will be introduced.

In this demo, you will create an inference endpoint URL that you can query to make predictions with a machine learning model classifier trained on the Iris Dataset.

The Iris flower data set consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters. Based on the combination of these four features, Fisher developed a linear discriminant model to distinguish the species from each other.

![Iris Dataset](./iris.png "Iris dataset")

See [sklearn-ONNX](http://onnx.ai/sklearn-onnx/)

## Install Python dependencies

In [1]:
%%bash

#apt-get install jq curl
#pip install -q scikit-learn numpy pandas boto3

## Train a model with scikit-learn

In [2]:
# Prepare model saving directory
import os
from pathlib import Path

SAVE_DIR = os.path.join(Path('./'), 'saved_models')
if not os.path.isdir(SAVE_DIR):
    os.makedirs(SAVE_DIR)
os.environ["SAVE_DIR"] = SAVE_DIR

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Load the dataset
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Explore the dataset
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = pd.Series(iris.target)
labels = iris.target_names
df['target_labels'] = labels[df.target]
df.sample(n=5, random_state=1).style

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_labels
14,5.800000,4.000000,1.200000,0.200000,0,setosa
98,5.100000,2.500000,3.000000,1.100000,1,versicolor
75,6.600000,3.000000,4.400000,1.400000,1,versicolor
16,5.400000,3.900000,1.300000,0.400000,0,setosa
131,7.900000,3.800000,6.400000,2.000000,2,virginica


In [4]:
# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [5]:
# Make some predictions with the test dataset
preds = model.predict(X_test)

# Display a few predictions
for i in range(5):
    print("Prediction: {} - Ground truth label {}".format(preds[i], y_test[i]))

Prediction: 1 - Ground truth label 1
Prediction: 2 - Ground truth label 1
Prediction: 0 - Ground truth label 0
Prediction: 1 - Ground truth label 1
Prediction: 2 - Ground truth label 2


## Convert the Scikit-Learn model to ONNX format

In [6]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType


FILE_PATH = os.path.join(SAVE_DIR, 'iris.onnx')

initial_type = [('float_input', FloatTensorType([None, 4]))]

onnx = convert_sklearn(model, initial_types=initial_type)
with open(FILE_PATH, "wb") as f:
    f.write(onnx.SerializeToString())
    
print("Model saved in {}\n".format(FILE_PATH))


# Check the model
import onnx
onnx_model = onnx.load(Path(FILE_PATH))
onnx.checker.check_model(onnx_model)


# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession(FILE_PATH)
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onnx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]

# Display a few predictions
for i in range(5):
    print("Prediction: {} - Ground truth label {}".format(pred_onnx[i], y_test[i]))

Model saved in ./saved_models/iris.onnx

Prediction: 1 - Ground truth label 1
Prediction: 2 - Ground truth label 1
Prediction: 0 - Ground truth label 0
Prediction: 1 - Ground truth label 1
Prediction: 2 - Ground truth label 2


## Setup your Scaleway Credentials

````
# More information about environment variables for Scaleway at
# https://github.com/scaleway/scaleway-sdk-go/tree/master/scw#environment-variables
# For a new secret key, click on the button [Generate new API Key], at
# https://console.scaleway.com/project/credentials

import os

os.environ["SCW_ACCESS_KEY"] = "xxx"
os.environ["SCW_SECRET_KEY"] = "xxx"
os.environ["SCW_DEFAULT_ORGANIZATION_ID"] = "xxx"
os.environ["SCW_DEFAULT_PROJECT_ID"] = "xxx"

os.environ["SCW_DEFAULT_REGION"] = "fr-par"
os.environ["SCW_DEFAULT_ZONE"] = "fr-par-1"
os.environ["SCW_API_URL"] = "https://api.scaleway.com"

os.environ["SCW_ACCESS_KEY_S3"] = "xxx"
os.environ["SCW_SECRET_KEY_S3"] = "xxx"

os.environ["S3_BUCKET_NAME"] = "xxx"
````

In [7]:
# Load credentials through environment variables and setup constants

# Get your credentials at https://console.scaleway.com/project/credentials
# For a new secret key, click on the button [Generate new API Key].
# More information about environment variables for Scaleway at
# https://github.com/scaleway/scaleway-sdk-go/tree/master/scw#environment-variables
environment_variable_keys = [
    "SCW_ACCESS_KEY",
    "SCW_SECRET_KEY",
    "SCW_DEFAULT_ORGANIZATION_ID",
    "SCW_DEFAULT_PROJECT_ID",
    "SCW_DEFAULT_REGION",
    "SCW_DEFAULT_ZONE",
    "SCW_API_URL",
    "SCW_ACCESS_KEY_S3",
    "SCW_SECRET_KEY_S3",
    "S3_BUCKET_NAME"
]

def hiden_secret(value):
    str_ = ""
    start = 0
    middle = min(8, int(len(value)/2))
    end = len(value)
    for i in range(start, middle):
        str_ += value[i]
    for i in range(middle, end):
        if value[i] == "-":
            str_ += "-"
        else:
            str_ += "✴"
    return str_

def print_environment_variable(key):
    if key in ["SCW_ACCESS_KEY", "SCW_SECRET_KEY", "SCW_DEFAULT_ORGANIZATION_ID", "SCW_DEFAULT_PROJECT_ID", "SCW_ACCESS_KEY_S3", "SCW_SECRET_KEY_S3"]:
        value = hiden_secret(os.getenv(key))
    else:
        value = os.getenv(key)
    print(f'  ${key}: {value}')

def assert_environment_variable(key):
    assert os.getenv(key) is not None, f"no value for environment variable ${key}"

def validate_environment_variables(environment_variable_keys):
    print('Validating environment variables')
    for key in environment_variable_keys:
        print_environment_variable(key)
        assert_environment_variable(key)

def validate_access_key(access_key):
    len_access_key = len(access_key)
    expected_len = 20
    assert len_access_key == expected_len, f"""invalid length for access key
        access key: {access_key[:8]}...
        expected length: {expected_len}
        length:          {len_access_key}"""

def validate_secret_key(secret_key):
    len_secret_key = len(secret_key)
    expected_len = 36
    assert len_secret_key == expected_len, f"""invalid length for secret key
        secret key: {secret_key[:8]}...
        expected length: {expected_len}
        length:          {len_secret_key}"""
        
validate_environment_variables(environment_variable_keys)

# If you prefer not to pass your credentials through environment variables, 
# you can overide the following constants directly.
SCW_ACCESS_KEY = os.getenv("SCW_ACCESS_KEY")
SCW_SECRET_KEY = os.getenv("SCW_SECRET_KEY")
SCW_DEFAULT_ORGANIZATION_ID = os.getenv("SCW_DEFAULT_ORGANIZATION_ID")
SCW_DEFAULT_PROJECT_ID = os.getenv("SCW_DEFAULT_PROJECT_ID")
SCW_DEFAULT_REGION = os.getenv("SCW_DEFAULT_REGION")
SCW_DEFAULT_ZONE = os.getenv("SCW_DEFAULT_ZONE")
SCW_API_URL = os.getenv("SCW_API_URL")
SCW_ACCESS_KEY_S3 = os.getenv("SCW_ACCESS_KEY_S3")
SCW_SECRET_KEY_S3 = os.getenv("SCW_SECRET_KEY_S3")

S3_BUCKET_NAME = os.getenv("S3_BUCKET_NAME")

validate_access_key(SCW_ACCESS_KEY)
validate_secret_key(SCW_SECRET_KEY)


Validating environment variables
  $SCW_ACCESS_KEY: SCW73JGG✴✴✴✴✴✴✴✴✴✴✴✴
  $SCW_SECRET_KEY: dd631c2c-✴✴✴✴-✴✴✴✴-✴✴✴✴-✴✴✴✴✴✴✴✴✴✴✴✴
  $SCW_DEFAULT_ORGANIZATION_ID: 7c304503-✴✴✴✴-✴✴✴✴-✴✴✴✴-✴✴✴✴✴✴✴✴✴✴✴✴
  $SCW_DEFAULT_PROJECT_ID: 7c304503-✴✴✴✴-✴✴✴✴-✴✴✴✴-✴✴✴✴✴✴✴✴✴✴✴✴
  $SCW_DEFAULT_REGION: fr-par
  $SCW_DEFAULT_ZONE: fr-par-1
  $SCW_API_URL: https://api.scaleway.com
  $SCW_ACCESS_KEY_S3: SCW73JGG✴✴✴✴✴✴✴✴✴✴✴✴
  $SCW_SECRET_KEY_S3: dd631c2c-✴✴✴✴-✴✴✴✴-✴✴✴✴-✴✴✴✴✴✴✴✴✴✴✴✴
  $S3_BUCKET_NAME: inference-demo


## Upload the model in your Object Storage bucket

**Important** : The bucket name must be unique, so you have to customize it.

In [8]:
# Customize the Bucket Name
S3_BUCKET_NAME = f"inference-demo"

S3_BUCKET_FILENAME = "iris.onnx" 

S3_URL = f"https://s3.{SCW_DEFAULT_REGION}.scw.cloud"

SCW_BUCKET_CONSOLE_BASE_URL = f"https://console.scaleway.com/object-storage/buckets"
SCW_BUCKET_CONSOLE_URL_TEMPLATE = f"{SCW_BUCKET_CONSOLE_BASE_URL}/{SCW_DEFAULT_REGION}/{{bucket_name}}/explorer"


# We can upload our local model to Scaleway Object Storage (s3 bucket)

import sys
import boto3
from botocore.client import ClientError
import json
import requests

def bucket_exists(bucket):
    try:
        resource.meta.client.head_bucket(Bucket=bucket.name)
        return True
    except ClientError:
        return False

def create_bucket_if_not_exist(bucket_name):
    bucket = resource.Bucket(bucket_name)
    if bucket_exists(bucket) == False:
        print(f'creating bucket')
        # The Access policy needs to be public for both Bucket and object
        bucket.create(ACL="public-read")
    else:
        print(f'bucket already exists: {bucket_name}')
    return bucket

def upload_file(bucket, model_path, s3_bucket_filename):
    client = resource.meta.client
    obj_list = client.list_objects(Bucket=bucket.name)
    
    if "Contents" not in obj_list.keys():
        print(f'uploading file: {s3_bucket_filename}')
        with open(model_path, "rb") as model_file:
            bucket.upload_fileobj(model_file, s3_bucket_filename, ExtraArgs={'ACL':'public-read'})
    else:
        obj_key = [ key["Key"] for key in client.list_objects(Bucket=bucket.name)["Contents"] ]
        if s3_bucket_filename not in obj_key:
            print(f'uploading file: {s3_bucket_filename}')
            with open(model_path, "rb") as model_file:
                bucket.upload_fileobj(model_file, s3_bucket_filename, ExtraArgs={'ACL':'public-read'})
        else:
            print(f'file already exists: {s3_bucket_filename}')
    
    return f"{bucket.name}/{s3_bucket_filename}"


s3_session = boto3.Session(region_name=SCW_DEFAULT_REGION)

resource = s3_session.resource("s3",
    endpoint_url=S3_URL,
    aws_access_key_id=SCW_ACCESS_KEY_S3,
    aws_secret_access_key=SCW_SECRET_KEY_S3
)

bucket = create_bucket_if_not_exist(S3_BUCKET_NAME)

s3_model_path = upload_file(bucket, FILE_PATH, S3_BUCKET_FILENAME)

print(f's3_model_path:            {s3_model_path}')
object_storage_model_url = f'{S3_URL}/{s3_model_path}'
print(f'object_storage_model_url: {object_storage_model_url}')
print(f'console link:             {SCW_BUCKET_CONSOLE_URL_TEMPLATE.format(bucket_name=bucket.name)}')

bucket already exists: inference-demo
uploading file: iris.onnx
s3_model_path:            inference-demo/iris.onnx
object_storage_model_url: https://s3.fr-par.scw.cloud/inference-demo/iris.onnx
console link:             https://console.scaleway.com/object-storage/buckets/fr-par/inference-demo/explorer


## AI Inference - Create a model

### Make sure the Inference API is up

In [9]:
%%bash

# Make sure the Inference API is up
curl -s -X GET  -H "X-Auth-Token: $SCW_SECRET_KEY" \
                -G -d "project_id=$SCW_DEFAULT_PROJECT_ID" \
                https://api.scaleway.com/inference/v1alpha1 | jq '.'

{
  "name": "Inference",
  "description": "Scaleway Inference At Scale API",
  "version": "1.0.21",
  "documentation_url": "https://developer.scaleway.com/"
}


### List your Inference models deployed in production

**IMPORTANT** Make sure we are not over quotas : During the Early Access, each users (account) can launch up to 10 model inference endpoints.

In [10]:
%%bash

# List your Inference models deployed in production
curl -s -X GET  -H "X-Auth-Token: $SCW_SECRET_KEY" \
                -G -d "project_id=$SCW_DEFAULT_PROJECT_ID" \
                https://api.scaleway.com/inference/v1alpha1/models | jq '.'

{
  "models": [],
  "total_count": 0
}


If necessary, you can delete models (**Don't forget to fill the `MODEL_ID` before executing the next code cell**)



In [ ]:
%%bash

# Delete a model
export MODEL_ID="" # Update ID

curl -s -X DELETE  -H "X-Auth-Token: $SCW_SECRET_KEY" \
                -G -d "project_id=$SCW_DEFAULT_PROJECT_ID" \
                https://api.scaleway.com/inference/v1alpha1/models/$MODEL_ID | jq '.'

### Create a Inference model

In [11]:
%%bash

export MODEL_S3_URL="https://s3.fr-par.scw.cloud/inference-demo/iris.onnx"    # Update your model URL

payload()
{
  cat <<EOF
{
    "project_id": "$SCW_DEFAULT_PROJECT_ID",
    "name": "iris.onnx",
    "framework": "onnx:latest",
    "path": "$MODEL_S3_URL",
    "config": {"input_type": "float32"}    
}
EOF
}

curl -s -X POST 'https://api.scaleway.com/inference/v1alpha1/models' \
                -H 'X-Auth-Token: "'"$SCW_SECRET_KEY"'"' \
                -H 'Content-Type: application/json' \
                -d "$(payload)"


{"id":"3f3e790c-7652-453c-acb0-3ebd87685ed7","organization_id":"7c304503-a644-4364-9fa1-b75e0de3a5b6","status":"converting","path":"https://s3.fr-par.scw.cloud/inference-demo/iris.onnx","name":"iris.onnx","framework":"onnx:latest","config":{"input_type":"float32"},"endpoint":null,"error_message":null,"project_id":"7c304503-a644-4364-9fa1-b75e0de3a5b6","created_at":"2021-03-31T12:35:01.643036Z","updated_at":"2021-03-31T12:35:01.643036Z"}

Now we can check the status of the model. This can take 4-5 minutes, you may need to refresh the next cell several times until the status is "Ready". If everything goes fine, the status will change from "Converting", "Building", "Deploying" to "Ready".

In [12]:
%%bash

# List your Inference models deployed in production
curl -s -X GET  -H "X-Auth-Token: $SCW_SECRET_KEY" \
                -G -d "project_id=$SCW_DEFAULT_PROJECT_ID" \
                https://api.scaleway.com/inference/v1alpha1/models | jq '.'

{
  "models": [
    {
      "id": "3f3e790c-7652-453c-acb0-3ebd87685ed7",
      "organization_id": "7c304503-a644-4364-9fa1-b75e0de3a5b6",
      "status": "ready",
      "path": "https://s3.fr-par.scw.cloud/inference-demo/iris.onnx",
      "name": "iris.onnx",
      "framework": "onnx:latest",
      "config": {
        "input_type": "float32"
      },
      "endpoint": "https://inferenceatscaleprodbgq4euex-f3e790c-7652-453c-ac.functions.fnc.fr-par.scw.cloud",
      "error_message": null,
      "project_id": "7c304503-a644-4364-9fa1-b75e0de3a5b6",
      "created_at": "2021-03-31T12:35:01.643036Z",
      "updated_at": "2021-03-31T12:35:01.643036Z"
    }
  ],
  "total_count": 1
}


### Make a prediction

First take a data sample from the Irias test dataset

In [13]:
# Display a Sample from the test set
print("Data Features {} - Target Label {}".format(X_test[7], y_test[7]))


Data Features [6.3 2.7 4.9 1.8] - Target Label 2


And then make the API call :

IMPORTANT :
- Make sure to copy-paste the `ENDPOINT_URL`of the inference model
- Optionnaly you can chnage the input data values

In [14]:
%%bash

export ENDPOINT_URL="https://inferenceatscaleprodbgq4euex-f3e790c-7652-453c-ac.functions.fnc.fr-par.scw.cloud"   # Update the endpoint url

# Update Data values
curl -s X POST $ENDPOINT_URL \
               -H 'Content-Type: application/json' \
               -d '{
  "data": {
    "format": "array",
    "arrayData": {
      "shape": [1, 4],
      "values": [6.3, 2.7, 4.9, 1.8]  
    }
  }
}' 
                
        


{"data": {"format": "array", "arrayData": {"shape": [1], "values": [2]}}}


### Make predictions for a batch of 5 samples

In [15]:
# Display 5 samples from the test set
for i in range(5,10):
    print("Data Features {} - Target Label {}".format(X_test[i], y_test[i]))


Data Features [7.6 3.  6.6 2.1] - Target Label 2
Data Features [5.5 2.4 3.7 1. ] - Target Label 1
Data Features [6.3 2.7 4.9 1.8] - Target Label 2
Data Features [5.5 4.2 1.4 0.2] - Target Label 0
Data Features [5.1 3.5 1.4 0.3] - Target Label 0


And then make the API call :
    
IMPORTANT :
- Make sure to copy-paste the `ENDPOINT_URL`of teh inference model
- Optionnaly you can chnage the input data values

In [16]:
%%bash

export ENDPOINT_URL="https://inferenceatscaleprodbgq4euex-f3e790c-7652-453c-ac.functions.fnc.fr-par.scw.cloud"   # Update the endpoint url

# Update Data Values
curl -s X POST $ENDPOINT_URL \
               -H 'Content-Type: application/json' \
               -d '{
  "data": {
    "format": "array",
    "arrayData": {
      "shape": [5, 4],
      "values": [[7.6, 3.0, 6.6, 2.1], [5.5, 2.4, 3.7, 1.0], [6.3, 2.7, 4.9, 1.8], [5.5, 4.2, 1.4, 0.2], [5.1, 3.5, 1.4, 0.3]]
    }
  }
}' 
     

{"data": {"format": "array", "arrayData": {"shape": [5], "values": [2, 1, 2, 0, 0]}}}
